In [ ]:
import pickle
import pandas as pd

train_data = pd.read_csv(r'train.csv')
val_data = pd.read_csv(r'val.csv')
test_data = pd.read_csv(r'test.csv')
test_data.head()

,Image_name,Image_URL,OCR_extracted_text,corrected_text
0,chuck_chuck_norris_meme_10.jpg,https://gtmemes.com/wp-content/uploads/2019/03...,Some magicians can walk on water Chuck Norris...,Some magicians can walk on water Chuck Norris...
1,dr_evil_NDBB96K.png,https://i.imgur.com/NDBB96K.png,ONE MILLION DOLLARS made on imgur,ONE MILLION DOLLARS made on imgur
2,misog_2109e457d636565e2e06dce39874c5231e1.jpg,https://media0ch-a.akamaihd.net/83/96/9e457d63...,Me: Mom can my friend sleep over? Mom: That's ...,Me: Mom can my friend sleep over? Mom: That's ...
3,obama_2691536739_469698809820026_263513986_n.jpg,http://politicalmemes.com/wp-content/uploads/2...,THIS GUY INHERITED A MESS. DID HE WHINE ABOUT ...,THIS GUY INHERITED A MESS. DID HE WHINE ABOUT ...
4,kim_threat-kim-jong-un-allegedly-working-on-mu...,https://pics.me.me/threat-kim-jong-un-allegedl...,THREAT: Kim Jong Un allegedly working on multi...,THREAT: Kim Jong Un allegedly working on multi...


#Text - BiLSTM

In [ ]:
import pandas as pd
import numpy as np

train_data = pd.read_csv(r'train.csv')
val_data = pd.read_csv(r'val.csv')

#train_data=train_data.dropna()
#val_data=val_data.dropna()
train_data = train_data.reset_index(drop=True)
val_data = val_data.reset_index(drop=True)

import nltk
import re

clean=[]
for s in train_data.text_corrected:
    s = " ".join(re.split('[-_,.#@?()!:]',s))
    clean.append(s.lower())
train_data['Clean_Text'] = clean

clean=[]
for s in val_data.text_corrected:
    s = " ".join(re.split('[-_,.#@?()!:]',s))
    clean.append(s.lower())
val_data['Clean_Text'] = clean

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tk = Tokenizer(lower = True)
tk.fit_on_texts(train_data.Clean_Text.values)

X_train_seq = tk.texts_to_sequences(train_data.Clean_Text.values)
X_val_seq = tk.texts_to_sequences(val_data.Clean_Text.values)
X_train = pad_sequences(X_train_seq, maxlen=64, padding='post')
X_val = pad_sequences(X_val_seq, maxlen=64, padding='post')

train_data.head()

Using TensorFlow backend.


,image_name,text_ocr,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment,Clean_Text
0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,hilarious,general,not_offensive,not_motivational,very_positive,look there my friend lightyear now all sohalik...
1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,not_funny,general,not_offensive,motivational,very_positive,the best of 10 yearchallenge completed in le...
2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,very_funny,not_sarcastic,not_offensive,not_motivational,positive,sam thorne strippin follow follow saw every...
3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,very_funny,twisted_meaning,very_offensive,motivational,positive,10 year challenge sweet dee edition
4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,hilarious,very_twisted,very_offensive,not_motivational,neutral,10 year challenge with no filter 47 hilarious ...


In [ ]:
clean=[]
for s in test_data.corrected_text:
    s = " ".join(re.split('[-_,.#@?()!:]',s))
    clean.append(s.lower())
test_data['Clean_Text'] = clean

X_test_seq = tk.texts_to_sequences(test_data.Clean_Text.values)
X_test = pad_sequences(X_test_seq, maxlen=64, padding='post')

##A

In [ ]:
sent=[]
for s in train_data.overall_sentiment:
  if('positive' in s):
    sent.append('positive')
  elif('negative' in s):
    sent.append('negative')
  else:
    sent.append('neutral')    
train_data['Sentiment']=sent

sent=[]
for s in val_data.overall_sentiment:
  if('positive' in s):
    sent.append('positive')
  elif('negative' in s):
    sent.append('negative')
  else:
    sent.append('neutral')
val_data['Sentiment']=sent

y_train=train_data['Sentiment'].values
y_val=val_data['Sentiment'].values

from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

le = LabelEncoder()
y_train_le = le.fit_transform(y_train)
y_val_le = le.transform(y_val)
y_train_cat = to_categorical(y_train_le)
y_val_cat = to_categorical(y_val_le)

In [ ]:
val_data.Sentiment.value_counts()

positive    630
neutral     307
negative    112
Name: Sentiment, dtype: int64

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',np.unique(train_data.Sentiment.values),
                                                  train_data.Sentiment.values)
class_weights= {0:class_weights[0],1:class_weights[1],2:class_weights[2]}

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GRU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import datetime, os

Batch_size = 32
vocabulary_size = len(tk.word_counts.keys())+1
max_words = 64
embedding_size = 32
model = Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words,))
#                    weights=[embedding_matrix], trainable=False))
model.add(Dropout(0.5))
#model.add(Bidirectional(LSTM(4, dropout=0.5,recurrent_dropout=0, return_sequences=True)))
#model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(16))) #, dropout=0.5,recurrent_dropout=0.5, kernel_regularizer=l2(0.01) 
model.add(Dropout(0.5))
#model.add(Dense(3, activation='softmax', kernel_regularizer=l2(0.01)))
model.add(Dense(3, activation='softmax'))
#adam=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, decay=1e-2)
#sgd = keras.optimizers.SGD(lr=0.01, decay=1e-2, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

early=EarlyStopping(patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=0, verbose=1)

history=model.fit(X_train, y_train_cat, validation_data=(X_val, y_val_cat), batch_size=Batch_size,
          epochs=50, 
          class_weight=class_weights, 
          callbacks=[early,reduce_lr]
          )

Epoch 1/50
186/186 [==============================] - 3s 15ms/step - loss: 1.0987 - acc: 0.2780 - val_loss: 1.0941 - val_acc: 0.4032 - lr: 0.0100
Epoch 2/50
186/186 [==============================] - 2s 11ms/step - loss: 1.0996 - acc: 0.3306 - val_loss: 1.1001 - val_acc: 0.1935 - lr: 0.0100
Epoch 3/50
186/186 [==============================] - ETA: 0s - loss: 1.0987 - acc: 0.3120
Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0009999999776482583.
186/186 [==============================] - 2s 11ms/step - loss: 1.0987 - acc: 0.3120 - val_loss: 1.0996 - val_acc: 0.2450 - lr: 0.0100
Epoch 4/50
186/186 [==============================] - 2s 11ms/step - loss: 1.0987 - acc: 0.3441 - val_loss: 1.0999 - val_acc: 0.2374 - lr: 1.0000e-03
Epoch 5/50
185/186 [============================>.] - ETA: 0s - loss: 1.1007 - acc: 0.3309
Epoch 00005: ReduceLROnPlateau reducing learning rate to 9.999999310821295e-05.
186/186 [==============================] - 2s 11ms/step - loss: 1.0991 - acc: 0.3

In [ ]:
y_train_prob=model.predict(X_train)
y_val_prob=model.predict(X_val)

y_train_pred = [np.argmax(x) for x in y_train_prob]
y_val_pred = [np.argmax(x) for x in y_val_prob]

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,auc

print("Training Accuracy : {0}%".format(int(100*accuracy_score(y_train_le, y_train_pred))))
print("Test Accuracy : {0}%\n".format(int(100*accuracy_score(y_val_le, y_val_pred))))
print(classification_report(y_val_le, y_val_pred))

Training Accuracy : 41%
Test Accuracy : 40%

              precision    recall  f1-score   support

           0       0.20      0.02      0.03       112
           1       0.30      0.71      0.42       307
           2       0.64      0.32      0.43       630

    accuracy                           0.40      1049
   macro avg       0.38      0.35      0.30      1049
weighted avg       0.50      0.40      0.38      1049



In [ ]:
y_test_prob=model.predict(X_test)
y_test_pred = [np.argmax(x) for x in y_test_prob]

with open('A.pkl', 'wb') as f:
  pickle.dump(y_test_pred, f)

##B

In [ ]:
y_train=[]
for i in train_data.index:
  labs=[]
  labs += [0] if train_data.at[i,'humour']=='not_funny' else [1]
  labs += [0] if train_data.at[i,'sarcasm']=='not_sarcastic' else [1]
  labs += [0] if train_data.at[i,'offensive']=='not_offensive' else [1]
  labs += [0] if train_data.at[i,'motivational']=='not_motivational' else [1]
  y_train.append(labs)

y_val=[]
for i in val_data.index:
  labs=[]
  labs += [0] if val_data.at[i,'humour']=='not_funny' else [1]
  labs += [0] if val_data.at[i,'sarcasm']=='not_sarcastic' else [1]
  labs += [0] if val_data.at[i,'offensive']=='not_offensive' else [1]
  labs += [0] if val_data.at[i,'motivational']=='not_motivational' else [1]
  y_val.append(labs)

y_train=np.array(y_train)
y_val=np.array(y_val)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GRU
from tensorflow.keras.regularizers import l2
import datetime, os
from sklearn.utils import class_weight

cls = 3   #Toggle 0,1,2,3 for humor, sarcasm, offensive, motivational respectively

class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train[:,cls]), y_train[:,cls])
class_weights= {0:class_weights[0],1:class_weights[1]}

Batch_size = 32
vocabulary_size = len(tk.word_counts.keys())+1
max_words = 64
embedding_size = 32
model = Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words,))
#                    weights=[embedding_matrix], trainable=False))
model.add(Dropout(0.5))
#model.add(Bidirectional(LSTM(4, dropout=0.5,recurrent_dropout=0, return_sequences=True)))
#model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(32)))#, dropout=0.5,recurrent_dropout=0.5, kernel_regularizer=l2(0.01))))
model.add(Dropout(0.5))
#model.add(Dense(3, activation='softmax', kernel_regularizer=l2(0.01)))
model.add(Dense(1, activation='sigmoid'))
#adam=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, decay=1e-2)
#sgd = keras.optimizers.SGD(lr=0.01, decay=1e-2, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

early=EarlyStopping(patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=0, verbose=1)

history=model.fit(X_train, y_train[:,cls], validation_data=(X_val, y_val[:,cls]), batch_size=Batch_size,
          epochs=50, 
          class_weight=class_weights,
          callbacks=[early,reduce_lr]
          )

Epoch 1/50
186/186 [==============================] - 4s 21ms/step - loss: 0.6936 - acc: 0.5304 - val_loss: 0.6932 - val_acc: 0.5091 - lr: 0.0010
Epoch 2/50
186/186 [==============================] - 2s 13ms/step - loss: 0.6868 - acc: 0.5558 - val_loss: 0.6790 - val_acc: 0.5624 - lr: 0.0010
Epoch 3/50
184/186 [============================>.] - ETA: 0s - loss: 0.6215 - acc: 0.6595
Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
186/186 [==============================] - 2s 13ms/step - loss: 0.6214 - acc: 0.6591 - val_loss: 0.7367 - val_acc: 0.5291 - lr: 0.0010
Epoch 4/50
186/186 [==============================] - ETA: 0s - loss: 0.4651 - acc: 0.7887
Epoch 00004: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
186/186 [==============================] - 2s 13ms/step - loss: 0.4651 - acc: 0.7887 - val_loss: 0.8081 - val_acc: 0.5291 - lr: 1.0000e-04
Epoch 5/50
185/186 [============================>.] - ETA: 0s - loss: 0.4307 - acc: 0.7926

In [ ]:
y_train_prob=model.predict(X_train)
y_val_prob=model.predict(X_val)

y_train_pred = [1 if x>0.5 else 0 for x in y_train_prob]
y_val_pred = [1 if x>0.5 else 0 for x in y_val_prob]

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,auc

print("Training Accuracy : {0}%".format(int(100*accuracy_score(y_train[:,cls], y_train_pred))))
print("Test Accuracy : {0}%\n".format(int(100*accuracy_score(y_val[:,cls], y_val_pred))))
print(classification_report(y_val[:,cls], y_val_pred))

Training Accuracy : 72%
Test Accuracy : 56%

              precision    recall  f1-score   support

           0       0.65      0.72      0.68       681
           1       0.34      0.27      0.30       368

    accuracy                           0.56      1049
   macro avg       0.50      0.50      0.49      1049
weighted avg       0.54      0.56      0.55      1049



In [ ]:
y_test_prob=model.predict(X_test)
y_test_pred = [1 if x>0.5 else 0 for x in y_test_prob]

with open('B'+str(cls)+'.pkl', 'wb') as f:
  pickle.dump(y_test_pred, f)

##C

In [ ]:
y_train=[]
for i in train_data.index:
  labs=[]
  x=1
  if train_data.at[i,'humour']=='not_funny':
    x=0
  elif train_data.at[i,'humour']=='very_funny':
    x=2
  elif train_data.at[i,'humour']=='hilarious':
    x=3
  labs+=[x]
  x=1
  if train_data.at[i,'sarcasm']=='not_sarcastic':
    x=0
  elif train_data.at[i,'sarcasm']=='twisted_meaning':
    x=2
  elif train_data.at[i,'sarcasm']=='very_twisted':
    x=3
  labs+=[x]
  x=1
  if train_data.at[i,'offensive']=='not_offensive':
    x=0
  elif train_data.at[i,'offensive']=='very_offensive':
    x=2
  elif train_data.at[i,'offensive']=='hateful_offensive':
    x=3
  labs+=[x]
  y_train.append(labs)

y_val=[]
for i in val_data.index:
  labs=[]
  x=1
  if val_data.at[i,'humour']=='not_funny':
    x=0
  elif val_data.at[i,'humour']=='very_funny':
    x=2
  elif val_data.at[i,'humour']=='hilarious':
    x=3
  labs+=[x]
  x=1
  if val_data.at[i,'sarcasm']=='not_sarcastic':
    x=0
  elif val_data.at[i,'sarcasm']=='twisted_meaning':
    x=2
  elif val_data.at[i,'sarcasm']=='very_twisted':
    x=3
  labs+=[x]
  x=1
  if val_data.at[i,'offensive']=='not_offensive':
    x=0
  elif val_data.at[i,'offensive']=='very_offensive':
    x=2
  elif val_data.at[i,'offensive']=='hateful_offensive':
    x=3
  labs+=[x]
  y_val.append(labs)

y_train=np.array(y_train)
y_val=np.array(y_val)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GRU
from tensorflow.keras.regularizers import l2
import datetime, os
from sklearn.utils import class_weight

cls = 2   #Toggle 0,1,2 for humor, sarcasm, offensive respectively

class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train[:,cls]), y_train[:,cls])
class_weights= {0:class_weights[0],1:class_weights[1],2:class_weights[2],3:class_weights[3]}

Batch_size = 32
vocabulary_size = len(tk.word_counts.keys())+1
max_words = 64
embedding_size = 32
model = Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words,))
#                    weights=[embedding_matrix], trainable=False))
model.add(Dropout(0.5))
#model.add(Bidirectional(LSTM(4, dropout=0.5,recurrent_dropout=0, return_sequences=True)))
#model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(16)))#, dropout=0.5,recurrent_dropout=0.5, kernel_regularizer=l2(0.01))))
model.add(Dropout(0.5))
#model.add(Dense(3, activation='softmax', kernel_regularizer=l2(0.01)))
model.add(Dense(4, activation='softmax'))
#adam=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, decay=1e-2)
#sgd = keras.optimizers.SGD(lr=0.01, decay=1e-2, momentum=0.9, nesterov=True)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])

history=model.fit(X_train, y_train[:,cls], validation_data=(X_val, y_val[:,cls]), batch_size=Batch_size,
          epochs=50, 
          class_weight=class_weights,
          callbacks=[early,reduce_lr]
          )

Epoch 1/50
186/186 [==============================] - 3s 17ms/step - loss: 1.3872 - acc: 0.2275 - val_loss: 1.3795 - val_acc: 0.3413 - lr: 0.0010
Epoch 2/50
184/186 [============================>.] - ETA: 0s - loss: 1.3831 - acc: 0.3606
Epoch 00002: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
186/186 [==============================] - 2s 13ms/step - loss: 1.3792 - acc: 0.3599 - val_loss: 1.3916 - val_acc: 0.2869 - lr: 0.0010
Epoch 3/50
186/186 [==============================] - 2s 13ms/step - loss: 1.3492 - acc: 0.4038 - val_loss: 1.3769 - val_acc: 0.3098 - lr: 1.0000e-04
Epoch 4/50
184/186 [============================>.] - ETA: 0s - loss: 1.3338 - acc: 0.4378
Epoch 00004: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
186/186 [==============================] - 2s 13ms/step - loss: 1.3327 - acc: 0.4363 - val_loss: 1.3790 - val_acc: 0.2974 - lr: 1.0000e-04
Epoch 5/50
186/186 [==============================] - 2s 13ms/step - loss: 1.3219 - ac

In [ ]:
y_train_prob=model.predict(X_train)
y_val_prob=model.predict(X_val)

y_train_pred = [np.argmax(x) for x in y_train_prob]
y_val_pred = [np.argmax(x) for x in y_val_prob]

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,auc

print("Training Accuracy : {0}%".format(int(100*accuracy_score(y_train[:,cls], y_train_pred))))
print("Test Accuracy : {0}%\n".format(int(100*accuracy_score(y_val[:,cls], y_val_pred))))
print(classification_report(y_val[:,cls], y_val_pred))

Training Accuracy : 52%
Test Accuracy : 32%

              precision    recall  f1-score   support

           0       0.41      0.48      0.44       424
           1       0.40      0.14      0.21       389
           2       0.20      0.36      0.26       207
           3       0.03      0.03      0.03        29

    accuracy                           0.32      1049
   macro avg       0.26      0.26      0.24      1049
weighted avg       0.35      0.32      0.31      1049



In [ ]:
y_test_prob=model.predict(X_test)
y_test_pred = [np.argmax(x) for x in y_test_prob]

with open('C'+str(cls)+'.pkl', 'wb') as f:
  pickle.dump(y_test_pred, f)

#Image - AlexNet

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/My Drive/"
!kaggle datasets download --unzip williamscott701/memotion-dataset-7k

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from keras.preprocessing import image
train_images=[]
val_images=[]

HEIGHT=224
WIDTH=224

for filename in val_data.image_name:
  img = image.load_img('/content/memotion_dataset_7k/images/'+filename, target_size=(HEIGHT, WIDTH))
  x = image.img_to_array(img)
  val_images.append(x/255.)

for filename in train_data.image_name:
  img = image.load_img('/content/memotion_dataset_7k/images/'+filename, target_size=(HEIGHT, WIDTH))
  x = image.img_to_array(img)
  train_images.append(x/255.)

train_images=np.array(train_images)
val_images=np.array(val_images)

In [ ]:
test_images=[]
for filename in test_data.Image_name:
  img = image.load_img('/content/test/'+filename, target_size=(HEIGHT, WIDTH))
  x = image.img_to_array(img)
  test_images.append(x/255.)

test_images=np.array(test_images)

##A

In [ ]:
sent=[]
for s in train_data.overall_sentiment:
  if('positive' in s):
    sent.append('positive')
  elif('negative' in s):
    sent.append('negative')
  else:
    sent.append('neutral')
    
train_data['Sentiment']=sent

sent=[]
for s in val_data.overall_sentiment:
  if('positive' in s):
    sent.append('positive')
  elif('negative' in s):
    sent.append('negative')
  else:
    sent.append('neutral')
    
val_data['Sentiment']=sent

y_train=train_data['Sentiment'].values
y_val=val_data['Sentiment'].values

from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

le = LabelEncoder()
y_train_le = le.fit_transform(y_train)
y_val_le = le.transform(y_val)
y_train_cat = to_categorical(y_train_le)
y_val_cat = to_categorical(y_val_le)

from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import optimizers
from tensorflow.keras.layers import BatchNormalization, ZeroPadding2D
from tensorflow.keras.regularizers import l2

img_shape=(224, 224, 3)
n_classes=3
l2_reg=0.
dp=0.3

model = Sequential()
# Layer 1
model.add(Conv2D(96, (11, 11), input_shape=img_shape,
  padding='same', kernel_regularizer=l2(l2_reg)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(dp))

# Layer 2
model.add(Conv2D(128, (5, 5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(dp))

# Layer 3
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(dp))

# Layer 4
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(dp))

# Layer 5
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(dp))

# Layer 6
model.add(Flatten())
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(dp))

# Layer 7
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(dp))

# Layer 8
model.add(Dense(n_classes))
model.add(BatchNormalization())
model.add(Activation('softmax'))

model.compile(optimizer='adam',
              loss=['categorical_crossentropy'],
              metrics=['accuracy'])

from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',np.unique(train_data.Sentiment.values),
                                                  train_data.Sentiment.values)
class_weights= {0:class_weights[0],1:class_weights[1],2:class_weights[2]}

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

EPOCHS = 50
BATCH_SIZE = 64
STEPS_PER_EPOCH = int(len(train_data)/BATCH_SIZE)+1
VALIDATION_STEPS = int(len(val_data)/BATCH_SIZE)+1

early=EarlyStopping(patience=5, restore_best_weights=False, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, verbose=1)

history = model.fit(
    train_images, y_train_cat,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=(val_images, y_val_cat),
    validation_steps=VALIDATION_STEPS,
    callbacks=[early,reduce_lr],
    class_weight=class_weights,
    )

Epoch 1/50
93/93 [==============================] - 27s 290ms/step - loss: 1.1292 - accuracy: 0.3202 - val_loss: 1.1423 - val_accuracy: 0.1068 - lr: 0.0010
Epoch 2/50
93/93 [==============================] - ETA: 0s - loss: 1.0972 - accuracy: 0.3286
Epoch 00002: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
93/93 [==============================] - 27s 288ms/step - loss: 1.0972 - accuracy: 0.3286 - val_loss: 1.1604 - val_accuracy: 0.2555 - lr: 0.0010
Epoch 3/50
93/93 [==============================] - 27s 288ms/step - loss: 1.0844 - accuracy: 0.3559 - val_loss: 1.1039 - val_accuracy: 0.2498 - lr: 1.0000e-04
Epoch 4/50
93/93 [==============================] - ETA: 0s - loss: 1.0764 - accuracy: 0.3591
Epoch 00004: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
93/93 [==============================] - 27s 288ms/step - loss: 1.0764 - accuracy: 0.3591 - val_loss: 1.1072 - val_accuracy: 0.2126 - lr: 1.0000e-04
Epoch 5/50
93/93 [======================

In [ ]:
y_train_prob=model.predict(train_images)
y_val_prob=model.predict(val_images)

y_train_pred = [np.argmax(x) for x in y_train_prob]
y_val_pred = [np.argmax(x) for x in y_val_prob]

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,auc

print("Training Accuracy : {0}%".format(int(100*accuracy_score(y_train_le, y_train_pred))))
print("Test Accuracy : {0}%\n".format(int(100*accuracy_score(y_val_le, y_val_pred))))
print(classification_report(y_val_le, y_val_pred))

Training Accuracy : 18%
Test Accuracy : 17%

              precision    recall  f1-score   support

           0       0.10      0.71      0.18       112
           1       0.27      0.18      0.22       307
           2       0.64      0.07      0.12       630

    accuracy                           0.17      1049
   macro avg       0.34      0.32      0.17      1049
weighted avg       0.48      0.17      0.16      1049



In [ ]:
y_test_prob=model.predict(test_images)
y_test_pred = [np.argmax(x) for x in y_test_prob]

with open('A.pkl', 'wb') as f:
  pickle.dump(y_test_pred, f)

##B

In [ ]:
y_train=[]
for i in train_data.index:
  labs=[]
  labs += [0] if train_data.at[i,'humour']=='not_funny' else [1]
  labs += [0] if train_data.at[i,'sarcasm']=='not_sarcastic' else [1]
  labs += [0] if train_data.at[i,'offensive']=='not_offensive' else [1]
  labs += [0] if train_data.at[i,'motivational']=='not_motivational' else [1]
  y_train.append(labs)

y_val=[]
for i in val_data.index:
  labs=[]
  labs += [0] if val_data.at[i,'humour']=='not_funny' else [1]
  labs += [0] if val_data.at[i,'sarcasm']=='not_sarcastic' else [1]
  labs += [0] if val_data.at[i,'offensive']=='not_offensive' else [1]
  labs += [0] if val_data.at[i,'motivational']=='not_motivational' else [1]
  y_val.append(labs)

y_train=np.array(y_train)
y_val=np.array(y_val)

In [ ]:
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import optimizers
from tensorflow.keras.layers import BatchNormalization, ZeroPadding2D
from tensorflow.keras.regularizers import l2

cls=3   #Toggle 0,1,2,3 for humor, sarcasm, offensive, motivational respectively

img_shape=(224, 224, 3)
n_classes=1
l2_reg=0.
dp=0.4

model = Sequential()
# Layer 1
model.add(Conv2D(96, (11, 11), input_shape=img_shape,
  padding='same', kernel_regularizer=l2(l2_reg)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(dp))

# Layer 2
model.add(Conv2D(128, (5, 5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(dp))

# Layer 3
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(dp))

# Layer 4
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(dp))

# Layer 5
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(dp))

# Layer 6
model.add(Flatten())
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(dp))

# Layer 7
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(dp))

# Layer 8
model.add(Dense(n_classes))
model.add(BatchNormalization())
model.add(Activation('sigmoid'))

model.compile(optimizer='adam',
              loss=['binary_crossentropy'],
              metrics=['accuracy'])

from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train[:,cls]), y_train[:,cls])
class_weights= {0:class_weights[0],1:class_weights[1]}

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

EPOCHS = 50
STEPS_PER_EPOCH = int(len(train_data)/BATCH_SIZE)+1
VALIDATION_STEPS = int(len(val_data)/BATCH_SIZE)+1

early=EarlyStopping(patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, verbose=1)

history=model.fit(train_images, y_train[:,cls],
	validation_data=(val_images, y_val[:,cls]),
	epochs=50, batch_size=64, class_weight=class_weights, callbacks=[early, reduce_lr])

Epoch 1/50
93/93 [==============================] - 27s 289ms/step - loss: 0.7881 - accuracy: 0.4331 - val_loss: 1.1695 - val_accuracy: 0.5834 - lr: 0.0010
Epoch 2/50
93/93 [==============================] - 27s 288ms/step - loss: 0.7621 - accuracy: 0.4222 - val_loss: 0.6935 - val_accuracy: 0.5882 - lr: 0.0010
Epoch 3/50
93/93 [==============================] - ETA: 0s - loss: 0.7536 - accuracy: 0.4646
Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
93/93 [==============================] - 27s 286ms/step - loss: 0.7536 - accuracy: 0.4646 - val_loss: 0.8805 - val_accuracy: 0.5605 - lr: 0.0010
Epoch 4/50
93/93 [==============================] - ETA: 0s - loss: 0.7347 - accuracy: 0.5162
Epoch 00004: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
93/93 [==============================] - 27s 285ms/step - loss: 0.7347 - accuracy: 0.5162 - val_loss: 0.7392 - val_accuracy: 0.4728 - lr: 1.0000e-04
Epoch 5/50
93/93 [==========================

In [ ]:
y_train_prob=model.predict(train_images)
y_val_prob=model.predict(val_images)

y_train_pred = [1 if x>0.5 else 0 for x in y_train_prob]
y_val_pred = [1 if x>0.5 else 0 for x in y_val_prob]

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,auc

print("Training Accuracy : {0}%".format(int(100*accuracy_score(y_train[:,cls], y_train_pred))))
print("Test Accuracy : {0}%\n".format(int(100*accuracy_score(y_val[:,cls], y_val_pred))))
print(classification_report(y_val[:,cls], y_val_pred))

Training Accuracy : 60%
Test Accuracy : 58%

              precision    recall  f1-score   support

           0       0.65      0.79      0.71       681
           1       0.36      0.22      0.27       368

    accuracy                           0.59      1049
   macro avg       0.50      0.50      0.49      1049
weighted avg       0.55      0.59      0.56      1049



In [ ]:
y_test_prob=model.predict(test_images)
y_test_pred = [1 if x>0.5 else 0 for x in y_test_prob]

with open('B'+str(cls)+'.pkl', 'wb') as f:
  pickle.dump(y_test_pred, f)

##C

In [ ]:
y_train=[]
for i in train_data.index:
  labs=[]
  x=1
  if train_data.at[i,'humour']=='not_funny':
    x=0
  elif train_data.at[i,'humour']=='very_funny':
    x=2
  elif train_data.at[i,'humour']=='hilarious':
    x=3
  labs+=[x]
  x=1
  if train_data.at[i,'sarcasm']=='not_sarcastic':
    x=0
  elif train_data.at[i,'sarcasm']=='twisted_meaning':
    x=2
  elif train_data.at[i,'sarcasm']=='very_twisted':
    x=3
  labs+=[x]
  x=1
  if train_data.at[i,'offensive']=='not_offensive':
    x=0
  elif train_data.at[i,'offensive']=='very_offensive':
    x=2
  elif train_data.at[i,'offensive']=='hateful_offensive':
    x=3
  labs+=[x]
  y_train.append(labs)

y_val=[]
for i in val_data.index:
  labs=[]
  x=1
  if val_data.at[i,'humour']=='not_funny':
    x=0
  elif val_data.at[i,'humour']=='very_funny':
    x=2
  elif val_data.at[i,'humour']=='hilarious':
    x=3
  labs+=[x]
  x=1
  if val_data.at[i,'sarcasm']=='not_sarcastic':
    x=0
  elif val_data.at[i,'sarcasm']=='twisted_meaning':
    x=2
  elif val_data.at[i,'sarcasm']=='very_twisted':
    x=3
  labs+=[x]
  x=1
  if val_data.at[i,'offensive']=='not_offensive':
    x=0
  elif val_data.at[i,'offensive']=='very_offensive':
    x=2
  elif val_data.at[i,'offensive']=='hateful_offensive':
    x=3
  labs+=[x]
  y_val.append(labs)

y_train=np.array(y_train)
y_val=np.array(y_val)

In [ ]:
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import optimizers
from tensorflow.keras.layers import BatchNormalization, ZeroPadding2D
from tensorflow.keras.regularizers import l2

cls=2   #Toggle 0,1,2 for humor, sarcasm, offensive respectively

img_shape=(224, 224, 3)
n_classes=4
l2_reg=0.
dp=0.2

model = Sequential()
# Layer 1
model.add(Conv2D(96, (11, 11), input_shape=img_shape,
  padding='same', kernel_regularizer=l2(l2_reg)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(dp))

# Layer 2
model.add(Conv2D(128, (5, 5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(dp))

# Layer 3
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(dp))

# Layer 4
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(dp))

# Layer 5
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(dp))

# Layer 6
model.add(Flatten())
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(dp))

# Layer 7
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(dp))

# Layer 8
model.add(Dense(n_classes))
model.add(BatchNormalization())
model.add(Activation('softmax'))

model.compile(optimizer='adam',
              loss=['sparse_categorical_crossentropy'],
              metrics=['accuracy'])

from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train[:,cls]), y_train[:,cls])
class_weights= {0:class_weights[0],1:class_weights[1],2:class_weights[2],3:class_weights[3]}

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

EPOCHS = 10
STEPS_PER_EPOCH = int(len(train_data)/BATCH_SIZE)+1
VALIDATION_STEPS = int(len(val_data)/BATCH_SIZE)+1

early=EarlyStopping(patience=5, restore_best_weights=False, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, verbose=1)

history=model.fit(train_images, y_train[:,cls],
	validation_data=(val_images, y_val[:,cls]),
	epochs=50, batch_size=64, class_weight=class_weights, callbacks=[early, reduce_lr])

Epoch 1/50
93/93 [==============================] - 37s 397ms/step - loss: 1.4737 - accuracy: 0.1805 - val_loss: 1.4934 - val_accuracy: 0.1373 - lr: 0.0010
Epoch 2/50
93/93 [==============================] - 37s 394ms/step - loss: 1.3827 - accuracy: 0.2522 - val_loss: 1.3843 - val_accuracy: 0.2202 - lr: 0.0010
Epoch 3/50
93/93 [==============================] - ETA: 0s - loss: 1.3705 - accuracy: 0.2509
Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
93/93 [==============================] - 37s 394ms/step - loss: 1.3705 - accuracy: 0.2509 - val_loss: 1.3856 - val_accuracy: 0.2440 - lr: 0.0010
Epoch 4/50
93/93 [==============================] - ETA: 0s - loss: 1.3368 - accuracy: 0.2628
Epoch 00004: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
93/93 [==============================] - 37s 394ms/step - loss: 1.3368 - accuracy: 0.2628 - val_loss: 1.3946 - val_accuracy: 0.2164 - lr: 1.0000e-04
Epoch 5/50
93/93 [==========================

In [ ]:
y_train_prob=model.predict(train_images)
y_val_prob=model.predict(val_images)

y_train_pred = [np.argmax(x) for x in y_train_prob]
y_val_pred = [np.argmax(x) for x in y_val_prob]

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,auc

print("Training Accuracy : {0}%".format(int(100*accuracy_score(y_train[:,cls], y_train_pred))))
print("Test Accuracy : {0}%\n".format(int(100*accuracy_score(y_val[:,cls], y_val_pred))))
print(classification_report(y_val[:,cls], y_val_pred))

Training Accuracy : 22%
Test Accuracy : 17%

              precision    recall  f1-score   support

           0       0.39      0.13      0.20       424
           1       0.40      0.18      0.25       389
           2       0.20      0.22      0.21       207
           3       0.03      0.52      0.06        29

    accuracy                           0.18      1049
   macro avg       0.26      0.26      0.18      1049
weighted avg       0.35      0.18      0.21      1049



In [ ]:
y_test_prob=model.predict(test_images)
y_test_pred = [np.argmax(x) for x in y_test_prob]

with open('C'+str(cls)+'.pkl', 'wb') as f:
  pickle.dump(y_test_pred, f)

#Combined - BiLSTM+AlexNet

In [ ]:
import pandas as pd
import numpy as np

train_data = pd.read_csv(r'train.csv')
val_data = pd.read_csv(r'val.csv')

#train_data=train_data.dropna()
#val_data=val_data.dropna()
train_data = train_data.reset_index(drop=True)
val_data = val_data.reset_index(drop=True)

import nltk
import re

clean=[]
for s in train_data.text_corrected:
    s = " ".join(re.split('[-_,.#@?()!:]',s))
    clean.append(s.lower())
train_data['Clean_Text'] = clean

clean=[]
for s in val_data.text_corrected:
    s = " ".join(re.split('[-_,.#@?()!:]',s))
    clean.append(s.lower())
val_data['Clean_Text'] = clean

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tk = Tokenizer(lower = True)
tk.fit_on_texts(train_data.Clean_Text.values)

X_train_seq = tk.texts_to_sequences(train_data.Clean_Text.values)
X_val_seq = tk.texts_to_sequences(val_data.Clean_Text.values)
X_train = pad_sequences(X_train_seq, maxlen=64, padding='post')
X_val = pad_sequences(X_val_seq, maxlen=64, padding='post')

clean=[]
for s in test_data.corrected_text:
    s = " ".join(re.split('[-_,.#@?()!:]',s))
    clean.append(s.lower())
test_data['Clean_Text'] = clean

X_test_seq = tk.texts_to_sequences(test_data.Clean_Text.values)
X_test = pad_sequences(X_test_seq, maxlen=64, padding='post')

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/My Drive/"
!kaggle datasets download --unzip williamscott701/memotion-dataset-7k

 98% 681M/695M [00:15<00:00, 44.8MB/s]
100% 695M/695M [00:15<00:00, 45.8MB/s]


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from keras.preprocessing import image
train_images=[]
val_images=[]

HEIGHT=224
WIDTH=224

for filename in val_data.image_name:
  img = image.load_img('/content/memotion_dataset_7k/images/'+filename, target_size=(HEIGHT, WIDTH))
  x = image.img_to_array(img)
  val_images.append(x/255.)

for filename in train_data.image_name:
  img = image.load_img('/content/memotion_dataset_7k/images/'+filename, target_size=(HEIGHT, WIDTH))
  x = image.img_to_array(img)
  train_images.append(x/255.)

train_images=np.array(train_images)
val_images=np.array(val_images)

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [ ]:
test_images=[]
for filename in test_data.Image_name:
  img = image.load_img('/content/test/'+filename, target_size=(HEIGHT, WIDTH))
  x = image.img_to_array(img)
  test_images.append(x/255.)

test_images=np.array(test_images)

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


##A

In [ ]:
sent=[]
for s in train_data.overall_sentiment:
  if('positive' in s):
    sent.append('positive')
  elif('negative' in s):
    sent.append('negative')
  else:
    sent.append('neutral')    
train_data['Sentiment']=sent

sent=[]
for s in val_data.overall_sentiment:
  if('positive' in s):
    sent.append('positive')
  elif('negative' in s):
    sent.append('negative')
  else:
    sent.append('neutral')
val_data['Sentiment']=sent

y_train=train_data['Sentiment'].values
y_val=val_data['Sentiment'].values

from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

le = LabelEncoder()
y_train_le = le.fit_transform(y_train)
y_val_le = le.transform(y_val)
y_train_cat = to_categorical(y_train_le)
y_val_cat = to_categorical(y_val_le)

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, Activation, ZeroPadding2D
from tensorflow.keras.layers import Input,Dense,Bidirectional,Conv2D,MaxPooling2D,Flatten,concatenate,GlobalAveragePooling2D,BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',np.unique(train_data.Sentiment.values),
                                                  train_data.Sentiment.values)
class_weights= {0:class_weights[0],1:class_weights[1],2:class_weights[2]}

vocabulary_size = len(tk.word_counts.keys())+1
max_words = 64
embedding_size = 32
dp=0.4

txt_input = Input(shape=(64,))
x = Embedding(vocabulary_size, embedding_size, input_length=max_words,)(txt_input)
x = Dropout(dp)(x)
x = Bidirectional(LSTM(32))(x)#, dropout=0.4,recurrent_dropout=0.4, kernel_regularizer=l2(0.01)))(x)
x = Dropout(dp)(x)
x = Dense(64)(x)
x = Model(inputs=txt_input, outputs=x)

img_input = Input(shape=(224,224,3))
y = Conv2D(96, (11, 11), padding='same')(img_input)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D(pool_size=(2, 2))(y)
y = Dropout(dp)(y)

y = Conv2D(128, (5, 5), padding='same')(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D(pool_size=(2, 2))(y)
y = Dropout(dp)(y)

y = ZeroPadding2D((1, 1))(y)
y = Conv2D(128, (3, 3), padding='same')(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D(pool_size=(2, 2))(y)
y = Dropout(dp)(y)

y = ZeroPadding2D((1, 1))(y)
y = Conv2D(256, (3, 3), padding='same')(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D(pool_size=(2, 2))(y)
y = Dropout(dp)(y)

y = ZeroPadding2D((1, 1))(y)
y = Conv2D(256, (3, 3), padding='same')(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D(pool_size=(2, 2))(y)
y = Dropout(dp)(y)

y = Flatten()(y)
y = Dense(512)(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = Dropout(dp)(y)
y = Dense(64)(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)

y = Model(inputs=img_input, outputs=y)

shared_layer = Dense(64)
x_sh = shared_layer(x.output)
y_sh = shared_layer(y.output)

combined = concatenate([x_sh,y_sh])#[x.output, y.output])
z = Dense(128, activation="relu")(combined)
z = Dense(3, activation="softmax")(z)

model = Model(inputs=[x.input, y.input], outputs=z)
#adam=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, decay=1e-2)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
#model.summary()
early=EarlyStopping(patience=5, restore_best_weights=False, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, verbose=1)

history=model.fit([X_train, train_images], y_train_cat,
	validation_data=([X_val, val_images], y_val_cat),
	epochs=30, batch_size=32, class_weight=class_weights,callbacks=[early,reduce_lr])

Epoch 1/30
186/186 [==============================] - 26s 139ms/step - loss: 1.1324 - acc: 0.3123 - val_loss: 0.9784 - val_acc: 0.5958 - lr: 0.0010
Epoch 2/30
186/186 [==============================] - ETA: 0s - loss: 1.1122 - acc: 0.3290
Epoch 00002: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
186/186 [==============================] - 25s 134ms/step - loss: 1.1122 - acc: 0.3290 - val_loss: 1.1181 - val_acc: 0.2641 - lr: 0.0010
Epoch 3/30
186/186 [==============================] - ETA: 0s - loss: 1.0086 - acc: 0.4038
Epoch 00003: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
186/186 [==============================] - 25s 134ms/step - loss: 1.0086 - acc: 0.4038 - val_loss: 1.1143 - val_acc: 0.2946 - lr: 1.0000e-04
Epoch 4/30
186/186 [==============================] - ETA: 0s - loss: 0.9629 - acc: 0.4079
Epoch 00004: ReduceLROnPlateau reducing learning rate to 1.0000000656873453e-06.
186/186 [==============================] - 25s 134ms/step 

In [ ]:
y_train_prob=model.predict([X_train,train_images])
y_val_prob=model.predict([X_val,val_images])

y_train_pred = [np.argmax(x) for x in y_train_prob]
y_val_pred = [np.argmax(x) for x in y_val_prob]

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,auc

print("Training Accuracy : {0}%".format(int(100*accuracy_score(y_train_le, y_train_pred))))
print("Test Accuracy : {0}%\n".format(int(100*accuracy_score(y_val_le, y_val_pred))))
print(classification_report(y_val_le, y_val_pred))

Training Accuracy : 48%
Test Accuracy : 32%

              precision    recall  f1-score   support

           0       0.09      0.23      0.13       112
           1       0.29      0.39      0.34       307
           2       0.57      0.31      0.40       630

    accuracy                           0.33      1049
   macro avg       0.32      0.31      0.29      1049
weighted avg       0.44      0.33      0.35      1049



In [ ]:
y_test_prob=model.predict([X_test,test_images])
y_test_pred = [np.argmax(x) for x in y_test_prob]

with open('A.pkl', 'wb') as f:
  pickle.dump(y_test_pred, f)

##B

In [ ]:
y_train=[]
for i in train_data.index:
  labs=[]
  labs += [0] if train_data.at[i,'humour']=='not_funny' else [1]
  labs += [0] if train_data.at[i,'sarcasm']=='not_sarcastic' else [1]
  labs += [0] if train_data.at[i,'offensive']=='not_offensive' else [1]
  labs += [0] if train_data.at[i,'motivational']=='not_motivational' else [1]
  y_train.append(labs)

y_val=[]
for i in val_data.index:
  labs=[]
  labs += [0] if val_data.at[i,'humour']=='not_funny' else [1]
  labs += [0] if val_data.at[i,'sarcasm']=='not_sarcastic' else [1]
  labs += [0] if val_data.at[i,'offensive']=='not_offensive' else [1]
  labs += [0] if val_data.at[i,'motivational']=='not_motivational' else [1]
  y_val.append(labs)

y_train=np.array(y_train)
y_val=np.array(y_val)

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, Activation, ZeroPadding2D
from tensorflow.keras.layers import Input,Dense,Bidirectional,Conv2D,MaxPooling2D,Flatten,concatenate,GlobalAveragePooling2D,BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

cls=3   #Toggle 0,1,2,3 for humor, sarcasm, offensive, motivational respectively

from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train[:,cls]), y_train[:,cls])
class_weights= {0:class_weights[0],1:class_weights[1]}

vocabulary_size = len(tk.word_counts.keys())+1
max_words = 64
embedding_size = 32
dp=0.4

txt_input = Input(shape=(64,))
x = Embedding(vocabulary_size, embedding_size, input_length=max_words,)(txt_input)
x = Dropout(dp)(x)
x = Bidirectional(LSTM(32))(x)#, dropout=0.4,recurrent_dropout=0.4, kernel_regularizer=l2(0.01)))(x)
x = Dropout(dp)(x)
x = Dense(64)(x)
x = Model(inputs=txt_input, outputs=x)

img_input = Input(shape=(224,224,3))
y = Conv2D(96, (11, 11), padding='same')(img_input)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D(pool_size=(2, 2))(y)
y = Dropout(dp)(y)

y = Conv2D(128, (5, 5), padding='same')(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D(pool_size=(2, 2))(y)
y = Dropout(dp)(y)

y = ZeroPadding2D((1, 1))(y)
y = Conv2D(128, (3, 3), padding='same')(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D(pool_size=(2, 2))(y)
y = Dropout(dp)(y)

y = ZeroPadding2D((1, 1))(y)
y = Conv2D(256, (3, 3), padding='same')(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D(pool_size=(2, 2))(y)
y = Dropout(dp)(y)

y = ZeroPadding2D((1, 1))(y)
y = Conv2D(256, (3, 3), padding='same')(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D(pool_size=(2, 2))(y)
y = Dropout(dp)(y)

y = Flatten()(y)
y = Dense(512)(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = Dropout(dp)(y)
y = Dense(64)(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)

y = Model(inputs=img_input, outputs=y)

combined = concatenate([x.output, y.output])
z = Dense(128, activation="relu")(combined)
z = Dense(2, activation="softmax")(z)

model = Model(inputs=[x.input, y.input], outputs=z)
#adam=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, decay=1e-3)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])
#model.summary()
early=EarlyStopping(patience = 5, restore_best_weights=False, verbose=1)

history=model.fit([X_train, train_images], y_train[:,cls],
	validation_data=([X_val, val_images], y_val[:,cls]),
	epochs=20, batch_size=32, class_weight=class_weights, callbacks=[early,reduce_lr])

Epoch 1/20
186/186 [==============================] - 26s 137ms/step - loss: 0.7084 - acc: 0.5112 - val_loss: 0.6896 - val_acc: 0.5119 - lr: 0.0010
Epoch 2/20
186/186 [==============================] - ETA: 0s - loss: 0.6949 - acc: 0.5496
Epoch 00002: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
186/186 [==============================] - 25s 133ms/step - loss: 0.6949 - acc: 0.5496 - val_loss: 0.7828 - val_acc: 0.3794 - lr: 0.0010
Epoch 3/20
186/186 [==============================] - ETA: 0s - loss: 0.5672 - acc: 0.7089
Epoch 00003: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
186/186 [==============================] - 25s 133ms/step - loss: 0.5672 - acc: 0.7089 - val_loss: 0.7473 - val_acc: 0.5586 - lr: 1.0000e-04
Epoch 4/20
186/186 [==============================] - ETA: 0s - loss: 0.5043 - acc: 0.7594
Epoch 00004: ReduceLROnPlateau reducing learning rate to 1.0000000656873453e-06.
186/186 [==============================] - 25s 133ms/step 

In [ ]:
y_train_prob=model.predict([X_train,train_images])
y_val_prob=model.predict([X_val,val_images])

y_train_pred = [np.argmax(x) for x in y_train_prob]
y_val_pred = [np.argmax(x) for x in y_val_prob]

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,auc

print("Training Accuracy : {0}%".format(int(100*accuracy_score(y_train[:,cls], y_train_pred))))
print("Test Accuracy : {0}%\n".format(int(100*accuracy_score(y_val[:,cls], y_val_pred))))
print(classification_report(y_val[:,cls], y_val_pred))

Training Accuracy : 78%
Test Accuracy : 56%

              precision    recall  f1-score   support

           0       0.66      0.66      0.66       681
           1       0.38      0.38      0.38       368

    accuracy                           0.56      1049
   macro avg       0.52      0.52      0.52      1049
weighted avg       0.56      0.56      0.56      1049



In [ ]:
y_test_prob=model.predict([X_test,test_images])
y_test_pred = [np.argmax(x) for x in y_test_prob]

with open('B'+str(cls)+'.pkl', 'wb') as f:
  pickle.dump(y_test_pred, f)

##C

In [ ]:
y_train=[]
for i in train_data.index:
  labs=[]
  x=1
  if train_data.at[i,'humour']=='not_funny':
    x=0
  elif train_data.at[i,'humour']=='very_funny':
    x=2
  elif train_data.at[i,'humour']=='hilarious':
    x=3
  labs+=[x]
  x=1
  if train_data.at[i,'sarcasm']=='not_sarcastic':
    x=0
  elif train_data.at[i,'sarcasm']=='twisted_meaning':
    x=2
  elif train_data.at[i,'sarcasm']=='very_twisted':
    x=3
  labs+=[x]
  x=1
  if train_data.at[i,'offensive']=='not_offensive':
    x=0
  elif train_data.at[i,'offensive']=='very_offensive':
    x=2
  elif train_data.at[i,'offensive']=='hateful_offensive':
    x=3
  labs+=[x]
  y_train.append(labs)

y_val=[]
for i in val_data.index:
  labs=[]
  x=1
  if val_data.at[i,'humour']=='not_funny':
    x=0
  elif val_data.at[i,'humour']=='very_funny':
    x=2
  elif val_data.at[i,'humour']=='hilarious':
    x=3
  labs+=[x]
  x=1
  if val_data.at[i,'sarcasm']=='not_sarcastic':
    x=0
  elif val_data.at[i,'sarcasm']=='twisted_meaning':
    x=2
  elif val_data.at[i,'sarcasm']=='very_twisted':
    x=3
  labs+=[x]
  x=1
  if val_data.at[i,'offensive']=='not_offensive':
    x=0
  elif val_data.at[i,'offensive']=='very_offensive':
    x=2
  elif val_data.at[i,'offensive']=='hateful_offensive':
    x=3
  labs+=[x]
  y_val.append(labs)

y_train=np.array(y_train)
y_val=np.array(y_val)

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, Activation, ZeroPadding2D
from tensorflow.keras.layers import Input,Dense,Bidirectional,Conv2D,MaxPooling2D,Flatten,concatenate,GlobalAveragePooling2D,BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

cls=2   #Toggle 0,1,2 for humor, sarcasm, offensive respectively

from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train[:,cls]), y_train[:,cls])
class_weights= {0:class_weights[0],1:class_weights[1],2:class_weights[2],3:class_weights[3]}

vocabulary_size = len(tk.word_counts.keys())+1
max_words = 64
embedding_size = 32
dp=0.4

txt_input = Input(shape=(64,))
x = Embedding(vocabulary_size, embedding_size, input_length=max_words,)(txt_input)
x = Dropout(dp)(x)
x = Bidirectional(LSTM(32))(x)#, dropout=0.4,recurrent_dropout=0.4, kernel_regularizer=l2(0.01)))(x)
x = Dropout(dp)(x)
x = Dense(64)(x)
x = Model(inputs=txt_input, outputs=x)

img_input = Input(shape=(224,224,3))
y = Conv2D(96, (11, 11), padding='same')(img_input)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D(pool_size=(2, 2))(y)
y = Dropout(dp)(y)

y = Conv2D(128, (5, 5), padding='same')(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D(pool_size=(2, 2))(y)
y = Dropout(dp)(y)

y = ZeroPadding2D((1, 1))(y)
y = Conv2D(128, (3, 3), padding='same')(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D(pool_size=(2, 2))(y)
y = Dropout(dp)(y)

y = ZeroPadding2D((1, 1))(y)
y = Conv2D(256, (3, 3), padding='same')(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D(pool_size=(2, 2))(y)
y = Dropout(dp)(y)

y = ZeroPadding2D((1, 1))(y)
y = Conv2D(256, (3, 3), padding='same')(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D(pool_size=(2, 2))(y)
y = Dropout(dp)(y)

y = Flatten()(y)
y = Dense(512)(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = Dropout(dp)(y)
y = Dense(64)(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)

y = Model(inputs=img_input, outputs=y)

combined = concatenate([x.output, y.output])
z = Dense(128, activation="relu")(combined)
z = Dense(4, activation="softmax")(z)

model = Model(inputs=[x.input, y.input], outputs=z)
#adam=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, decay=1e-3)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])
#model.summary()
early=EarlyStopping(patience = 5, restore_best_weights=False, verbose=1)

history=model.fit([X_train, train_images], y_train[:,cls],
	validation_data=([X_val, val_images], y_val[:,cls]),
	epochs=20, batch_size=32, class_weight=class_weights, callbacks=[early, reduce_lr])

Epoch 1/20
186/186 [==============================] - 25s 137ms/step - loss: 1.4209 - acc: 0.2399 - val_loss: 1.4556 - val_acc: 0.1001 - lr: 0.0010
Epoch 2/20
186/186 [==============================] - 25s 133ms/step - loss: 1.4005 - acc: 0.2405 - val_loss: 1.4034 - val_acc: 0.1745 - lr: 0.0010
Epoch 3/20
186/186 [==============================] - 25s 132ms/step - loss: 1.3542 - acc: 0.2810 - val_loss: 1.3441 - val_acc: 0.2927 - lr: 0.0010
Epoch 4/20
186/186 [==============================] - ETA: 0s - loss: 1.0984 - acc: 0.4488
Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
186/186 [==============================] - 25s 132ms/step - loss: 1.0984 - acc: 0.4488 - val_loss: 1.3572 - val_acc: 0.3031 - lr: 0.0010
Epoch 5/20
186/186 [==============================] - ETA: 0s - loss: 0.7800 - acc: 0.6298
Epoch 00005: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
186/186 [==============================] - 25s 133ms/step - loss: 0.7800 - 

In [ ]:
y_train_prob=model.predict([X_train,train_images])
y_val_prob=model.predict([X_val,val_images])

y_train_pred = [np.argmax(x) for x in y_train_prob]
y_val_pred = [np.argmax(x) for x in y_val_prob]

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,auc

print("Training Accuracy : {0}%".format(int(100*accuracy_score(y_train[:,cls], y_train_pred))))
print("Test Accuracy : {0}%\n".format(int(100*accuracy_score(y_val[:,cls], y_val_pred))))
print(classification_report(y_val[:,cls], y_val_pred))

Training Accuracy : 72%
Test Accuracy : 30%

              precision    recall  f1-score   support

           0       0.37      0.33      0.35       424
           1       0.35      0.33      0.34       389
           2       0.21      0.24      0.23       207
           3       0.03      0.07      0.04        29

    accuracy                           0.30      1049
   macro avg       0.24      0.24      0.24      1049
weighted avg       0.32      0.30      0.31      1049



In [ ]:
y_test_prob=model.predict([X_test,test_images])
y_test_pred = [np.argmax(x) for x in y_test_prob]

with open('C'+str(cls)+'.pkl', 'wb') as f:
  pickle.dump(y_test_pred, f)

#Multitask - BiLSTM+AlexNet

In [ ]:
import pandas as pd
import numpy as np

train_data = pd.read_csv(r'train.csv')
val_data = pd.read_csv(r'val.csv')

#train_data=train_data.dropna()
#val_data=val_data.dropna()
train_data = train_data.reset_index(drop=True)
val_data = val_data.reset_index(drop=True)

import nltk
import re

clean=[]
for s in train_data.text_corrected:
    s = " ".join(re.split('[-_,.#@?()!:]',s))
    clean.append(s.lower())
train_data['Clean_Text'] = clean

clean=[]
for s in val_data.text_corrected:
    s = " ".join(re.split('[-_,.#@?()!:]',s))
    clean.append(s.lower())
val_data['Clean_Text'] = clean

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tk = Tokenizer(lower = True)
tk.fit_on_texts(train_data.Clean_Text.values)

X_train_seq = tk.texts_to_sequences(train_data.Clean_Text.values)
X_val_seq = tk.texts_to_sequences(val_data.Clean_Text.values)
X_train = pad_sequences(X_train_seq, maxlen=64, padding='post')
X_val = pad_sequences(X_val_seq, maxlen=64, padding='post')

clean=[]
for s in test_data.corrected_text:
    s = " ".join(re.split('[-_,.#@?()!:]',s))
    clean.append(s.lower())
test_data['Clean_Text'] = clean

X_test_seq = tk.texts_to_sequences(test_data.Clean_Text.values)
X_test = pad_sequences(X_test_seq, maxlen=64, padding='post')

test_images=[]
for filename in test_data.Image_name:
  img = image.load_img('/content/test/'+filename, target_size=(HEIGHT, WIDTH))
  x = image.img_to_array(img)
  test_images.append(x/255.)

test_images=np.array(test_images)

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/My Drive/"
!kaggle datasets download --unzip williamscott701/memotion-dataset-7k

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from keras.preprocessing import image
train_images=[]
val_images=[]

HEIGHT=224
WIDTH=224

for filename in val_data.image_name:
  img = image.load_img('/content/memotion_dataset_7k/images/'+filename, target_size=(HEIGHT, WIDTH))
  x = image.img_to_array(img)
  val_images.append(x/255.)

for filename in train_data.image_name:
  img = image.load_img('/content/memotion_dataset_7k/images/'+filename, target_size=(HEIGHT, WIDTH))
  x = image.img_to_array(img)
  train_images.append(x/255.)

train_images=np.array(train_images)
val_images=np.array(val_images)

In [ ]:
sent=[]
for s in train_data.overall_sentiment:
  if('positive' in s):
    sent.append('positive')
  elif('negative' in s):
    sent.append('negative')
  else:
    sent.append('neutral')    
train_data['Sentiment']=sent

sent=[]
for s in val_data.overall_sentiment:
  if('positive' in s):
    sent.append('positive')
  elif('negative' in s):
    sent.append('negative')
  else:
    sent.append('neutral')
val_data['Sentiment']=sent

y_train=train_data['Sentiment'].values
y_val=val_data['Sentiment'].values

from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

le = LabelEncoder()
y_train_le = le.fit_transform(y_train)
y_val_le = le.transform(y_val)
y_train_cat = to_categorical(y_train_le)
y_val_cat = to_categorical(y_val_le)

##A+B

In [ ]:
y_train=[]
for i in train_data.index:
  labs=[]
  labs += [0] if train_data.at[i,'humour']=='not_funny' else [1]
  labs += [0] if train_data.at[i,'sarcasm']=='not_sarcastic' else [1]
  labs += [0] if train_data.at[i,'offensive']=='not_offensive' else [1]
  labs += [0] if train_data.at[i,'motivational']=='not_motivational' else [1]
  y_train.append(labs)

y_val=[]
for i in val_data.index:
  labs=[]
  labs += [0] if val_data.at[i,'humour']=='not_funny' else [1]
  labs += [0] if val_data.at[i,'sarcasm']=='not_sarcastic' else [1]
  labs += [0] if val_data.at[i,'offensive']=='not_offensive' else [1]
  labs += [0] if val_data.at[i,'motivational']=='not_motivational' else [1]
  y_val.append(labs)

y_train=np.array(y_train)
y_val=np.array(y_val)

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, Activation, ZeroPadding2D
from tensorflow.keras.layers import Input,Dense,Bidirectional,Conv2D,MaxPooling2D,Flatten,concatenate,GlobalAveragePooling2D,BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.utils import class_weight
cw=[]
#class_weights = class_weight.compute_class_weight('balanced',np.unique(train_data.Sentiment.values),
#                                                  train_data.Sentiment.values)
#class_weights= {0:class_weights[0],1:class_weights[1],2:class_weights[2]}
#cw.append(class_weights)
for cls in (0,1,2,3):
  class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train[:,cls]), y_train[:,cls])
  class_weights= {0:class_weights[0],1:class_weights[1]}
  cw.append(class_weights)

vocabulary_size = len(tk.word_counts.keys())+1
max_words = 64
embedding_size = 32
dp=0.0

txt_input = Input(shape=(64,))
x = Embedding(vocabulary_size, embedding_size, input_length=max_words,)(txt_input)
x = Dropout(dp)(x)
x = Bidirectional(LSTM(32, recurrent_dropout=dp, kernel_regularizer=l2(0.01), return_sequences=True))(x)
x = Dropout(dp)(x)
x = Bidirectional(LSTM(32, recurrent_dropout=dp, kernel_regularizer=l2(0.01)))(x)
x = Dropout(dp)(x)
x = Dense(128, activation="relu")(x)
x = Model(inputs=txt_input, outputs=x)

img_input = Input(shape=(224,224,3))
y = Conv2D(96, (11, 11), padding='same')(img_input)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D(pool_size=(2, 2))(y)
y = Dropout(dp)(y)

y = Conv2D(128, (5, 5), padding='same')(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D(pool_size=(2, 2))(y)
y = Dropout(dp)(y)

y = ZeroPadding2D((1, 1))(y)
y = Conv2D(128, (3, 3), padding='same')(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D(pool_size=(2, 2))(y)
y = Dropout(dp)(y)

y = ZeroPadding2D((1, 1))(y)
y = Conv2D(256, (3, 3), padding='same')(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D(pool_size=(2, 2))(y)
y = Dropout(dp)(y)

y = ZeroPadding2D((1, 1))(y)
y = Conv2D(256, (3, 3), padding='same')(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D(pool_size=(2, 2))(y)
y = Dropout(dp)(y)

y = Flatten()(y)
y = Dense(512)(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = Dropout(dp)(y)
y = Dense(128)(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)

y = Model(inputs=img_input, outputs=y)

shared_layer = Dense(256, activation="relu")
x_sh = shared_layer(x.output)
y_sh = shared_layer(y.output)

combined = concatenate([x_sh, y_sh])#x.output, y.output])
z = Dense(256, activation="relu")(combined)

#task_a = Dense(128, activation="relu")(z)
#task_a = Dense(3, activation="softmax")(task_a)
task_b0 = Dense(128, activation="relu")(z)
task_b0 = Dense(2, activation="softmax")(task_b0)
task_b1 = Dense(128, activation="relu")(z)
task_b1 = Dense(2, activation="softmax")(task_b1)
task_b2 = Dense(128, activation="relu")(z)
task_b2 = Dense(2, activation="softmax")(task_b2)
task_b3 = Dense(128, activation="relu")(z)
task_b3 = Dense(2, activation="softmax")(task_b3)

model = Model(inputs=[x.input, y.input], outputs=[task_b0,task_b1,task_b2,task_b3])
#adam=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, decay=1e-3)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'], loss_weights=[1,1,1,1])

#model.load_weights('model.h5')
early=EarlyStopping(patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, verbose=1)

history=model.fit([X_train, train_images], [y_train[:,0],y_train[:,1],y_train[:,2],y_train[:,3]],
	validation_data=([X_val, val_images], [y_val[:,0],y_val[:,1],y_val[:,2],y_val[:,3]]),
	epochs=20, batch_size=32, class_weight=cw, 
  callbacks=[early,reduce_lr]
  )

#model.save_weights('model.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 5943 samples, validate on 1049 samples
Epoch 1/20
5943/5943 [==============================] - 88s 15ms/sample - loss: 3.5755 - dense_6_loss: 0.6996 - dense_8_loss: 0.6989 - dense_10_loss: 0.6992 - dense_12_loss: 0.7011 - dense_6_acc: 0.4774 - dense_8_acc: 0.4831 - dense_10_acc: 0.4891 - dense_12_acc: 0.4947 - val_loss: 2.7734 - val_dense_6_l

In [ ]:
y_train_prob=model.predict([X_train,train_images])
y_val_prob=model.predict([X_val,val_images])

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,auc
'''
y_train_pred = [np.argmax(x) for x in y_train_prob[0]]
y_val_pred = [np.argmax(x) for x in y_val_prob[0]]
print("Training Accuracy : {0}%".format(int(100*accuracy_score(y_train_le, y_train_pred))))
print("Test Accuracy : {0}%\n".format(int(100*accuracy_score(y_val_le, y_val_pred))))
print(classification_report(y_val_le, y_val_pred))
'''
for i in (0,1,2,3):
  print("Sub-task:"+str(i))
  y_train_pred = [np.argmax(x) for x in y_train_prob[i]]
  y_val_pred = [np.argmax(x) for x in y_val_prob[i]]
  print("Training Accuracy : {0}%".format(int(100*accuracy_score(y_train[:,i], y_train_pred))))
  print("Test Accuracy : {0}%\n".format(int(100*accuracy_score(y_val[:,i], y_val_pred))))
  print(classification_report(y_val[:,i], y_val_pred))

Sub-task:0
Training Accuracy : 62%
Test Accuracy : 60%

              precision    recall  f1-score   support

           0       0.22      0.25      0.23       251
           1       0.75      0.71      0.73       798

    accuracy                           0.60      1049
   macro avg       0.48      0.48      0.48      1049
weighted avg       0.62      0.60      0.61      1049

Sub-task:1
Training Accuracy : 75%
Test Accuracy : 74%

              precision    recall  f1-score   support

           0       0.21      0.05      0.08       240
           1       0.77      0.95      0.85       809

    accuracy                           0.74      1049
   macro avg       0.49      0.50      0.47      1049
weighted avg       0.64      0.74      0.67      1049

Sub-task:2
Training Accuracy : 59%
Test Accuracy : 56%

              precision    recall  f1-score   support

           0       0.39      0.11      0.18       424
           1       0.59      0.88      0.71       625

    accuracy  

In [ ]:
y_test_prob=model.predict([X_test,test_images])
'''
y_test_pred = [np.argmax(x) for x in y_test_prob[0]]
with open('A.pkl', 'wb') as f:
  pickle.dump(y_test_pred, f)
'''
for i in (0,1,2,3):
  y_test_pred = [np.argmax(x) for x in y_test_prob[i]] #!!!
  with open('B'+str(i)+'.pkl', 'wb') as f:
    pickle.dump(y_test_pred, f)

##A+C

In [ ]:
y_train=[]
for i in train_data.index:
  labs=[]
  x=1
  if train_data.at[i,'humour']=='not_funny':
    x=0
  elif train_data.at[i,'humour']=='very_funny':
    x=2
  elif train_data.at[i,'humour']=='hilarious':
    x=3
  labs+=[x]
  x=1
  if train_data.at[i,'sarcasm']=='not_sarcastic':
    x=0
  elif train_data.at[i,'sarcasm']=='twisted_meaning':
    x=2
  elif train_data.at[i,'sarcasm']=='very_twisted':
    x=3
  labs+=[x]
  x=1
  if train_data.at[i,'offensive']=='not_offensive':
    x=0
  elif train_data.at[i,'offensive']=='very_offensive':
    x=2
  elif train_data.at[i,'offensive']=='hateful_offensive':
    x=3
  labs+=[x]
  y_train.append(labs)

y_val=[]
for i in val_data.index:
  labs=[]
  x=1
  if val_data.at[i,'humour']=='not_funny':
    x=0
  elif val_data.at[i,'humour']=='very_funny':
    x=2
  elif val_data.at[i,'humour']=='hilarious':
    x=3
  labs+=[x]
  x=1
  if val_data.at[i,'sarcasm']=='not_sarcastic':
    x=0
  elif val_data.at[i,'sarcasm']=='twisted_meaning':
    x=2
  elif val_data.at[i,'sarcasm']=='very_twisted':
    x=3
  labs+=[x]
  x=1
  if val_data.at[i,'offensive']=='not_offensive':
    x=0
  elif val_data.at[i,'offensive']=='very_offensive':
    x=2
  elif val_data.at[i,'offensive']=='hateful_offensive':
    x=3
  labs+=[x]
  y_val.append(labs)

y_train=np.array(y_train)
y_val=np.array(y_val)

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, Activation, ZeroPadding2D
from tensorflow.keras.layers import Input,Dense,Bidirectional,Conv2D,MaxPooling2D,Flatten,concatenate,GlobalAveragePooling2D,BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.utils import class_weight
cw=[]
#class_weights = class_weight.compute_class_weight('balanced',np.unique(train_data.Sentiment.values),
#                                                  train_data.Sentiment.values)
#class_weights= {0:class_weights[0],1:class_weights[1],2:class_weights[2]}
#cw.append(class_weights)
for cls in (0,1,2):
  class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train[:,cls]), y_train[:,cls])
  class_weights= {0:class_weights[0],1:class_weights[1],2:class_weights[2],3:class_weights[3]}
  cw.append(class_weights)

vocabulary_size = len(tk.word_counts.keys())+1
max_words = 64
embedding_size = 32
dp=0

txt_input = Input(shape=(64,))
x = Embedding(vocabulary_size, embedding_size, input_length=max_words,)(txt_input)
x = Dropout(dp)(x)
x = Bidirectional(LSTM(32, recurrent_dropout=dp, kernel_regularizer=l2(0.01), return_sequences=True))(x)
x = Dropout(dp)(x)
x = Bidirectional(LSTM(32, recurrent_dropout=dp, kernel_regularizer=l2(0.01)))(x)
x = Dropout(dp)(x)
x = Dense(128, activation="relu")(x)
x = Model(inputs=txt_input, outputs=x)

img_input = Input(shape=(224,224,3))
y = Conv2D(96, (11, 11), padding='same')(img_input)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D(pool_size=(2, 2))(y)
y = Dropout(dp)(y)

y = Conv2D(128, (5, 5), padding='same')(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D(pool_size=(2, 2))(y)
y = Dropout(dp)(y)

y = ZeroPadding2D((1, 1))(y)
y = Conv2D(128, (3, 3), padding='same')(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D(pool_size=(2, 2))(y)
y = Dropout(dp)(y)

y = ZeroPadding2D((1, 1))(y)
y = Conv2D(256, (3, 3), padding='same')(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D(pool_size=(2, 2))(y)
y = Dropout(dp)(y)

y = ZeroPadding2D((1, 1))(y)
y = Conv2D(256, (3, 3), padding='same')(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D(pool_size=(2, 2))(y)
y = Dropout(dp)(y)

y = Flatten()(y)
y = Dense(512)(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = Dropout(dp)(y)
y = Dense(128)(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)

y = Model(inputs=img_input, outputs=y)

shared_layer = Dense(256, activation="relu")
x_sh = shared_layer(x.output)
y_sh = shared_layer(y.output)

combined = concatenate([x_sh, y_sh])#x.output, y.output])
z = Dense(256, activation="relu")(combined)

#task_a = Dense(3, activation="softmax")(z)
task_c0 = Dense(128, activation="relu")(z)
task_c0 = Dense(4, activation="softmax")(task_c0)
task_c1 = Dense(128, activation="relu")(z)
task_c1 = Dense(4, activation="softmax")(task_c1)
task_c2 = Dense(128, activation="relu")(z)
task_c2 = Dense(4, activation="softmax")(task_c2)

model = Model(inputs=[x.input, y.input], outputs=[task_c0,task_c1,task_c2])
#adam=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, decay=1e-3)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'], loss_weights=[1,1,1])
#model.summary()
early=EarlyStopping(patience=5, restore_best_weights=False, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, verbose=1)

history=model.fit([X_train, train_images], [y_train[:,0],y_train[:,1],y_train[:,2]],
	validation_data=([X_val, val_images], [y_val[:,0],y_val[:,1],y_val[:,2]]),
	epochs=20, batch_size=32, class_weight=cw, callbacks=[early,reduce_lr])

Train on 5943 samples, validate on 1049 samples
Epoch 1/20
5943/5943 [==============================] - 87s 15ms/sample - loss: 4.9748 - dense_30_loss: 1.3971 - dense_32_loss: 1.3964 - dense_34_loss: 1.4046 - dense_30_acc: 0.2315 - dense_32_acc: 0.2495 - dense_34_acc: 0.2305 - val_loss: 4.2021 - val_dense_30_loss: 1.3813 - val_dense_32_loss: 1.3842 - val_dense_34_loss: 1.3770 - val_dense_30_acc: 0.2669 - val_dense_32_acc: 0.2269 - val_dense_34_acc: 0.3699
Epoch 2/20
5943/5943 [==============================] - 82s 14ms/sample - loss: 4.1804 - dense_30_loss: 1.3887 - dense_32_loss: 1.3888 - dense_34_loss: 1.3924 - dense_30_acc: 0.2558 - dense_32_acc: 0.2717 - dense_34_acc: 0.2463 - val_loss: 4.1814 - val_dense_30_loss: 1.4138 - val_dense_32_loss: 1.3948 - val_dense_34_loss: 1.3730 - val_dense_30_acc: 0.1678 - val_dense_32_acc: 0.1735 - val_dense_34_acc: 0.3203
Epoch 3/20
5920/5943 [============================>.] - ETA: 0s - loss: 4.1642 - dense_30_loss: 1.3867 - dense_32_loss: 1.3873 -

In [ ]:
y_train_prob=model.predict([X_train,train_images])
y_val_prob=model.predict([X_val,val_images])

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,auc
'''
y_train_pred = [np.argmax(x) for x in y_train_prob[0]]
y_val_pred = [np.argmax(x) for x in y_val_prob[0]]
print("Training Accuracy : {0}%".format(int(100*accuracy_score(y_train_le, y_train_pred))))
print("Test Accuracy : {0}%\n".format(int(100*accuracy_score(y_val_le, y_val_pred))))
print(classification_report(y_val_le, y_val_pred))
'''
for i in (0,1,2):
  print("Sub-task:"+str(i))
  y_train_pred = [np.argmax(x) for x in y_train_prob[i]]
  y_val_pred = [np.argmax(x) for x in y_val_prob[i]]
  print("Training Accuracy : {0}%".format(int(100*accuracy_score(y_train[:,i], y_train_pred))))
  print("Test Accuracy : {0}%\n".format(int(100*accuracy_score(y_val[:,i], y_val_pred))))
  print(classification_report(y_val[:,i], y_val_pred))

Sub-task:0
Training Accuracy : 22%
Test Accuracy : 20%

              precision    recall  f1-score   support

           0       0.21      0.14      0.16       251
           1       0.32      0.17      0.22       392
           2       0.29      0.18      0.22       286
           3       0.14      0.57      0.22       120

    accuracy                           0.21      1049
   macro avg       0.24      0.26      0.21      1049
weighted avg       0.26      0.21      0.21      1049

Sub-task:1
Training Accuracy : 26%
Test Accuracy : 24%

              precision    recall  f1-score   support

           0       0.24      0.33      0.28       240
           1       0.52      0.27      0.35       524
           2       0.20      0.07      0.11       224
           3       0.06      0.39      0.11        61

    accuracy                           0.25      1049
   macro avg       0.26      0.27      0.21      1049
weighted avg       0.36      0.25      0.27      1049

Sub-task:2
Trainin

In [ ]:
y_test_prob=model.predict([X_test,test_images])
'''
y_test_pred = [np.argmax(x) for x in y_test_prob[0]]
with open('A.pkl', 'wb') as f:
  pickle.dump(y_test_pred, f)
'''
for i in (0,1,2):
  y_test_pred = [np.argmax(x) for x in y_test_prob[i]] #!!!
  with open('C'+str(i)+'.pkl', 'wb') as f:
    pickle.dump(y_test_pred, f)

#Test Submission

In [ ]:
import pickle
with open('A.pkl', 'rb') as A, open('B0.pkl', 'rb') as B0, open('B1.pkl', 'rb') as B1, open('B2.pkl', 'rb') as B2, open('B3.pkl', 'rb') as B3, open('C0.pkl', 'rb') as C0, open('C1.pkl', 'rb') as C1, open('C2.pkl', 'rb') as C2:
  a = pickle.load(A)
  b0 = pickle.load(B0)
  b1 = pickle.load(B1)
  b2 = pickle.load(B2)
  b3 = pickle.load(B3)
  c0 = pickle.load(C0)
  c1 = pickle.load(C1)
  c2 = pickle.load(C2)

#w = (np.max(y_test_prob[:,1])-0.5)/3
#c3=[0 if i<0.5 else int((i-0.5)/w)+1 for i in y_test_prob[:,1]]
c3=b3

out=[]
for i in range(1878):
  out.append(str(a[i]-1)+'_'+str(b0[i])+str(b1[i])+str(b2[i])+str(b3[i])+'_'+str(c0[i])+str(c1[i])+str(c2[i])+str(c3[i])+'\n')

with open('answer.txt', 'w') as f:
  f.writelines(out)

#!zip res.zip answer.txt

In [ ]:
y_test_prob

array([[0.5188661 , 0.48113385],
       [0.5129394 , 0.4870606 ],
       [0.4712976 , 0.5287024 ],
       ...,
       [0.49387637, 0.50612366],
       [0.43651247, 0.5634875 ],
       [0.40774596, 0.59225404]], dtype=float32)

In [ ]:
np.min(y_test_prob[:,1])

0.35645983